In [ ]:
import torch
import torch.nn.functional as F 
from transformers import AutoTokenizer

import sys
sys.path.append("..")

from bert_model import BERTClassifier, preprocess_text

In [ ]:
def load_bert_model(model_path="../final_aita_model.pth"):
    """Load and initialize the BERT model"""
    # Load the saved model with map_location to ensure compatibility with CPU
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
    model_name = checkpoint.get('model_name', 'roberta-base')
    label_encoder = checkpoint['label_encoder']
    print('Loaded model:', model_name)
    
    # Initialize model
    model = BERTClassifier(model_name=model_name, num_classes=len(label_encoder.classes_))
    model.load_state_dict(checkpoint['model_state_dict'])
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    
    # Also load and initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return {
        'model': model,
        'tokenizer': tokenizer,
        'label_encoder': label_encoder,
        'device': device
    }

def predict_with_model(text, loaded_model):
    """Make predictions with already loaded model"""
    model = loaded_model['model']
    tokenizer = loaded_model['tokenizer']
    label_encoder = loaded_model['label_encoder']
    device = loaded_model['device']
    
    # Preprocess text
    preprocessed_text = preprocess_text(text)
    
    # Tokenize
    encodings = tokenizer(
        preprocessed_text, 
        truncation=True, 
        padding='max_length', 
        max_length=256, 
        return_tensors='pt'
    )
    
    # Move to device
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    
    # Handle token_type_ids if present
    token_type_ids = encodings.get('token_type_ids')
    if token_type_ids is not None:
        token_type_ids = token_type_ids.to(device)
        outputs = model(input_ids, attention_mask, token_type_ids)
    else:
        outputs = model(input_ids, attention_mask)
    
    # Get prediction
    probs = F.softmax(outputs, dim=1)
    confidence, prediction = torch.max(probs, dim=1)
    
    verdict = label_encoder.inverse_transform([prediction.item()])[0]
    confidence = confidence.item()
    
    # Get all class probabilities
    all_probs = probs[0].cpu().detach().numpy()
    class_probs = {label_encoder.inverse_transform([i])[0]: float(prob) 
                  for i, prob in enumerate(all_probs)}
    
    return {
        'verdict': verdict,
        'confidence': confidence,
        'class_probabilities': class_probs
    }

In [ ]:
post = '''AITA for telling my stepson that we will not be providing him with free childcare and housing?
I (38F) have 3 stepsons, they are all adults (late teens-mid twenties). The one this concerns is middle child. He sat my husband (42) and I down last night and told us that his girlfriend is pregnant and they do plan on keeping the baby. We got into logistics of how that's going to work because they live with her mom right now in her living room. He said that they won't be able to stay with her once the baby is born.

This is where he said that they plan on moving in with us. As a statement. We do not have any free rooms, so I asked where they expected to stay? He said we can finish the basement before the baby is born and they will live down there. The basement is completely unfinished. Making it a livable space would cost several THOUSANDS of dollars - which we don't have.

He said it would just be until the baby reaches pre-school age and then it will be easier for them to live on their own since they wouldn't need the full time childcare. What childcare?? Me apparently. I'm childcare. I do work from home, but it is very much WORK. I spend 75% of my day in meetings or on phone calls with clients, it is not a job I can do with a baby. I'll admit, not my finest moment, but I laughed. Hard. Because I couldn't believe he was being serious.

My husband stepped in and said he needed to come up with a different plan because none of that was happening. He said that we can help financially to secure an apartment. We can help with baby stuff. We will be happy to take the baby when they need a break - but that's the extent. It wasn't a happy parting at the end, but my husband and I are on the same page.

A couple hours later my husband got a call from his ex. She was upset at both of us for not supporting them the way they felt they needed us to. Bear in mind, she can't watch the baby or house them either because she lives in a 1 bed apartment with her on again/off again husband who does not like the kids and does not allow them to even come over to see her. Hubby told her that now would be a great time for her and son/DIL to get a place together - then she can take care of the baby and they'll have a place to live. Which of course, she didn't like.

We don't really care what she thinks, but our oldest has said that he thinks it's a little harsh that we won't let them stay here temporarily (it wouldn't be temporary. Her mom's living room was supposed to be temporary 2 years ago.) and he doesn't think it was nice that i laughed during a serious discussion, which is fair. The youngest is on our side,but he lives at home when he's not away at college and he doesn't want to live with a baby either, so he's a little biased.

Are we assholes here? I do feel bad, it isn't going to be easy - but being a parent is never easy. Hell, that's the whole point of this freaking post, our kids are grown and it's STILL hard to be a parent. We'd love some outside insight.'''

In [ ]:
# Load the model
model = load_bert_model('../final_aita_model.pth')

# Make predictions as needed
prediction1 = predict_with_model(post, model)
print(prediction1)